In [1]:
import numpy as np
import pandas as pd

In [2]:
from functions_tracking import resolution_voxels, view_process_label_object, tracking_object

In [3]:
# Load in example image converted to numpy array

stack = np.load("stack_tracking_example.npy")

In [4]:
# Extract condensates

track_cost_cutoff = 130**2
splitting_cost_cutoff = 10**2
threshold = 16500
y_res = 0.0339  # from image metadata
x_res = 0.0339  # from image metadata
nr_images = stack.shape[0]

track_df, viewer, labels_per_frame = tracking_object(
    stack, track_cost_cutoff, splitting_cost_cutoff, threshold, y_res, x_res, nr_images
)

/opt/anaconda3/envs/mechcell/lib/python3.9/site-packages/morphometrics/measure/label.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27


In [5]:
# Check track ids from visualization

# Condensate 10:

track_id_4 = track_df.loc[(track_df["track_id"] == 4) & track_df["frame"].between(0, 10)]
track_id_22 = track_df.loc[(track_df["track_id"] == 22) & track_df["frame"].between(11, 23)]
track_id_43 = track_df.loc[(track_df["track_id"] == 43) & track_df["frame"].isin([24, 25])]
track_id_72 = track_df.loc[(track_df["track_id"] == 72) & (track_df["frame"] == 26)]

track_df_cond_10 = pd.concat([track_id_4, track_id_22, track_id_43, track_id_72])
track_df_cond_10 = track_df_cond_10.sort_values(by="frame")
track_df_cond_10.insert(5, "Condensate ID", 10)

# Condensate 11:

track_df_cond_11 = track_df.loc[(track_df["track_id"] == 5) & track_df["frame"].between(1, 27)]
track_df_cond_11.insert(5, "Condensate ID", 11)

# Condensate 12:

track_df_cond_12 = track_df.loc[(track_df["track_id"] == 7) & track_df["frame"].between(1, 17)]
track_df_cond_12.insert(5, "Condensate ID", 12)

# Condensate 13:

track_id_2 = track_df.loc[(track_df["track_id"] == 2) & track_df["frame"].isin([1, 2])]
track_id_10 = track_df.loc[(track_df["track_id"] == 10) & track_df["frame"].isin([3, 5])]
track_id_3 = track_df.loc[(track_df["track_id"] == 3) & (track_df["frame"] == 4)]
track_id_17 = track_df.loc[(track_df["track_id"] == 17) & track_df["frame"].isin([10, 11, 12])]
track_id_19 = track_df.loc[(track_df["track_id"] == 19) & (track_df["frame"] == 13)]

track_df_cond_13 = pd.concat([track_id_2, track_id_10, track_id_3, track_id_17, track_id_19])
track_df_cond_13 = track_df_cond_13.sort_values(by="frame")
track_df_cond_13.insert(5, "Condensate ID", 13)


# Combined to nucleus
track_df = pd.concat([track_df_cond_10, track_df_cond_11, track_df_cond_12, track_df_cond_13])
track_df.insert(6, "Nucleus ID", 3)

track_df

,label,centroid_y,centroid_x,frame,pixel_size,Condensate ID,Nucleus ID,track_id
4,5,813.773842,277.824251,0,734,10,3,4
12,8,819.442029,241.509662,1,828,10,3,4
19,7,820.118220,218.944367,2,719,10,3,4
30,11,821.402926,236.031915,3,752,10,3,4
44,14,820.831633,247.391156,4,588,10,3,4
...,...,...,...,...,...,...,...,...
52,8,672.562937,290.199301,5,286,13,3,10
141,16,684.740385,284.150641,10,312,13,3,17
162,13,678.512586,291.405034,11,437,13,3,17
191,21,682.963636,274.994805,12,385,13,3,17


In [6]:
# visualize tracks


viewer.add_tracks(
    track_df[["Condensate ID", "frame", "centroid_x", "centroid_y"]],
    # graph=graph,
    scale=(y_res, x_res),
)

new_labels = np.zeros_like(labels_per_frame)

for i, row in track_df.iterrows():
    frame = int(row["frame"])
    inds = labels_per_frame[frame] == row["label"]
    new_labels[frame][inds] = int(row["Condensate ID"]) + 1

viewer.add_labels(new_labels, scale=(y_res, x_res))

<Labels layer 'new_labels [1]' at 0x1d1a280d0>

In [7]:
# Extract nuclei

track_cost_cutoff = 130**2
splitting_cost_cutoff = 10**2
threshold_nuc = 5500

track_df_nuc, viewer, labels_per_frame = tracking_object(
    stack, track_cost_cutoff, splitting_cost_cutoff, threshold_nuc, y_res, x_res, nr_images
)
track_df_nuc.insert(6, "Nucleus ID", 3)

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27


In [9]:
merged = track_df_nuc.merge(track_df, on=["Nucleus ID", "frame"])


# rename for condensates and nuclei

merged = merged.rename(
    columns={
        "centroid_y_x": "centroid-y_nuc",
        "centroid_x_x": "centroid-x_nuc",
        "centroid_y_y": "centroid-y_cond",
        "centroid_x_y": "centroid-x_cond",
        "pixel_size_y": "area_cond",
        "track_id_y": "track_id",
    }
)

# adjust for nuclei movement for condensates
merged["y_corr"] = merged["centroid-y_cond"] - merged["centroid-y_nuc"]
merged["x_corr"] = merged["centroid-x_cond"] - merged["centroid-x_nuc"]

# scale the coordinates
merged["y_corr"] = merged["y_corr"] * y_res
merged["x_corr"] = merged["x_corr"] * x_res


merged = merged[["Condensate ID", "x_corr", "y_corr", "frame", "area_cond", "track_id", "Nucleus ID"]]
merged.insert(7, "Condition", "Confined")
merged.insert(8, "Video", "3626")

merged

,Condensate ID,x_corr,y_corr,frame,area_cond,track_id,Nucleus ID,Condition,Video
0,10,-0.574969,6.495878,0,734,4,3,Confined,3626
1,10,-1.165382,6.374272,1,828,4,3,Confined,3626
2,11,-0.229811,-6.908069,1,381,5,3,Confined,3626
3,12,2.748592,5.129694,1,59,7,3,Confined,3626
4,13,-0.692096,1.563245,1,378,2,3,Confined,3626
...,...,...,...,...,...,...,...,...,...
77,10,-0.554698,5.443862,25,592,43,3,Confined,3626
78,11,-1.330697,-5.487754,25,275,5,3,Confined,3626
79,10,-1.010359,5.189859,26,594,72,3,Confined,3626
80,11,-1.451170,-5.525722,26,229,5,3,Confined,3626


In [10]:
merged.to_csv("data_tracking_condensates.csv", index=False, header=True)